# 🎟️ 9.3 — Laboratorio: Sistema de Tickets con SQLite

En este laboratorio aplicarás todo lo aprendido sobre **bases de datos SQLite** para construir un pequeño sistema de gestión de tickets de soporte.

Aprenderás a:
- Crear varias tablas relacionadas.
- Insertar y consultar datos con claves foráneas.
- Ejecutar consultas SQL avanzadas.
- Mostrar resultados con Pandas.

💡 *Objetivo:* modelar una base de datos de soporte técnico con usuarios, categorías y tickets.

In [1]:
import sqlite3
import pandas as pd

conexion = sqlite3.connect('../../datasets/tickets.db')
cursor = conexion.cursor()
print('✅ Conectado a tickets.db')

✅ Conectado a tickets.db


---
## 1️⃣ Crear las tablas del sistema de tickets

Tendremos 3 tablas:

- **usuarios**: guarda el nombre y correo de los usuarios.
- **categorias**: agrupa los tipos de incidencias (hardware, software, red...).
- **tickets**: almacena cada solicitud de soporte con referencias a usuario y categoría.

In [2]:
cursor.executescript('''
DROP TABLE IF EXISTS tickets;
DROP TABLE IF EXISTS usuarios;
DROP TABLE IF EXISTS categorias;

CREATE TABLE usuarios (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL
);

CREATE TABLE categorias (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT NOT NULL
);

CREATE TABLE tickets (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    titulo TEXT NOT NULL,
    descripcion TEXT,
    estado TEXT DEFAULT 'abierto',
    fecha TEXT,
    usuario_id INTEGER,
    categoria_id INTEGER,
    FOREIGN KEY (usuario_id) REFERENCES usuarios(id),
    FOREIGN KEY (categoria_id) REFERENCES categorias(id)
);
''')

conexion.commit()
print('🧱 Tablas creadas correctamente.')

🧱 Tablas creadas correctamente.


---
## 2️⃣ Insertar datos de ejemplo

Poblamos las tablas con algunos usuarios, categorías y tickets iniciales.

In [3]:
usuarios = [
    ('Ana Pérez', 'ana@example.com'),
    ('Luis Gómez', 'luis@example.com'),
    ('Marta Ruiz', 'marta@example.com')
]

categorias = [
    ('Hardware',),
    ('Software',),
    ('Red',)
]

tickets = [
    ('No enciende el portátil', 'El equipo no arranca tras la última actualización', 'abierto', '2025-10-29', 1, 1),
    ('Error al abrir Word', 'El programa se bloquea al abrir documentos .docx', 'en proceso', '2025-10-30', 2, 2),
    ('Sin conexión a internet', 'El router parpadea en rojo', 'cerrado', '2025-10-28', 3, 3)
]

cursor.executemany('INSERT INTO usuarios (nombre, email) VALUES (?, ?)', usuarios)
cursor.executemany('INSERT INTO categorias (nombre) VALUES (?)', categorias)
cursor.executemany('INSERT INTO tickets (titulo, descripcion, estado, fecha, usuario_id, categoria_id) VALUES (?, ?, ?, ?, ?, ?)', tickets)

conexion.commit()
print('💾 Datos iniciales insertados correctamente.')

💾 Datos iniciales insertados correctamente.


---
## 3️⃣ Consultas básicas

Listamos las tablas para verificar que los datos están insertados correctamente.

In [4]:
print('👥 Usuarios:')
display(pd.read_sql_query('SELECT * FROM usuarios', conexion))

print('🏷️ Categorías:')
display(pd.read_sql_query('SELECT * FROM categorias', conexion))

print('🎫 Tickets:')
display(pd.read_sql_query('SELECT * FROM tickets', conexion))

👥 Usuarios:


,id,nombre,email
0,1,Ana Pérez,ana@example.com
1,2,Luis Gómez,luis@example.com
2,3,Marta Ruiz,marta@example.com


🏷️ Categorías:


,id,nombre
0,1,Hardware
1,2,Software
2,3,Red


🎫 Tickets:


,id,titulo,descripcion,estado,fecha,usuario_id,categoria_id
0,1,No enciende el portátil,El equipo no arranca tras la última actualización,abierto,2025-10-29,1,1
1,2,Error al abrir Word,El programa se bloquea al abrir documentos .docx,en proceso,2025-10-30,2,2
2,3,Sin conexión a internet,El router parpadea en rojo,cerrado,2025-10-28,3,3


---
## 4️⃣ 🧩 Ejercicio — Consulta con JOIN

Obtén un listado de tickets con:
- título
- estado
- fecha
- nombre del usuario
- nombre de la categoría

👉 *Pista:* utiliza `INNER JOIN` para unir las tres tablas.

In [5]:
# ✏️ Tu código aquí...

### ✅ Solución propuesta

In [29]:
query = '''
SELECT t.id, t.titulo, t.estado, t.fecha,
       u.nombre AS usuario, c.nombre AS categoria
FROM tickets t
INNER JOIN usuarios u ON t.usuario_id = u.id
INNER JOIN categorias c ON t.categoria_id = c.id
ORDER BY t.fecha DESC;
'''

df = pd.read_sql_query(query, conexion)
df



,id,titulo,estado,fecha,usuario,categoria
0,2,Error al abrir Word,en proceso,2025-10-30,Luis Gómez,Software
1,1,No enciende el portátil,cerrado,2025-10-29,Ana Pérez,Hardware


In [6]:
import sys


In [28]:
df_tickets = pd.read_sql_query("SELECT * FROM tickets", conexion)
df_usuarios = pd.read_sql_query("SELECT * FROM usuarios", conexion)
df_categorias = pd.read_sql_query("SELECT * FROM categorias", conexion)

df = df_tickets.merge(
    df_usuarios[['id','nombre']],
    left_on='usuario_id',
    right_on='id',
    how='inner'
).rename(columns={'nombre':'usuario'}).drop(columns=['id_y'])

df = df.merge(
    df_categorias[['id','nombre']], 
    left_on='categoria_id',
    right_on='id',
    how='inner'
).rename(columns={'nombre':'categoria'}).drop(columns=['id_x','usuario_id', 'categoria_id', 'id','descripcion'])

display(df)

,titulo,estado,fecha,usuario,categoria
0,No enciende el portátil,cerrado,2025-10-29,Ana Pérez,Hardware
1,Error al abrir Word,en proceso,2025-10-30,Luis Gómez,Software


---
## 5️⃣ 🧩 Ejercicio — Actualizar y eliminar

1. Actualiza el estado de un ticket a **cerrado**.
2. Elimina un ticket por su ID.

👉 *Pista:* usa `UPDATE` y `DELETE` con `WHERE`.

In [7]:
# ✏️ Tu código aquí...

### ✅ Solución propuesta

In [27]:
cursor.execute('UPDATE tickets SET estado = "cerrado" WHERE id = 1')
cursor.execute('DELETE FROM tickets WHERE id = 3')
conexion.commit()

print('✅ Cambios aplicados correctamente.')

✅ Cambios aplicados correctamente.


---
## 6️⃣ Consultar el estado final de los tickets

Comprobamos los cambios realizados.

In [30]:
df_final = pd.read_sql_query('SELECT * FROM tickets', conexion)
df_final

,id,titulo,descripcion,estado,fecha,usuario_id,categoria_id
0,1,No enciende el portátil,El equipo no arranca tras la última actualización,cerrado,2025-10-29,1,1
1,2,Error al abrir Word,El programa se bloquea al abrir documentos .docx,en proceso,2025-10-30,2,2


---
## 7️⃣ 🧠 Conclusiones

- Has modelado un sistema relacional simple con 3 tablas.
- Has trabajado con claves foráneas y consultas `JOIN`.
- Has practicado inserciones, actualizaciones y eliminaciones.
- Este mismo patrón se aplica en **PostgreSQL, MySQL o SQL Server** con ligeros ajustes.

💡 Próximo paso: integrar este modelo con una **API Flask o FastAPI** para exponer los datos como endpoints REST.

In [31]:
conexion.close()
print('🔒 Conexión cerrada correctamente.')

🔒 Conexión cerrada correctamente.
